In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

data = pd.read_csv('C:\\Users\\Pooja\\Downloads\\AML\\Ensemble\\Ensemble\\rainfall.csv')
X = data[['MinTemp','MaxTemp','WindSpeed','Humidity9am','RainToday']]
Y = data[['RainTomorrow']]

# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)

test_df = pd.concat ([X_test,y_test],axis =1 )

dt_m = DecisionTreeClassifier(random_state = 42)
dt_m.fit(X_train, y_train)
dt_pred = dt_m.predict(X_test)
test_df['dt_pred'] = dt_pred
test_df.to_csv('classification_results.csv')
score = accuracy_score(y_test, dt_pred)
score 

0.7076271186440678

In [2]:
from sklearn.ensemble import RandomForestClassifier

# creating a RF classifier
clf = RandomForestClassifier(random_state = 42) 
clf.fit(X_train, y_train)

rf_pred = clf.predict(X_test)
test_df['rf_pred'] = rf_pred
test_df.to_csv('classification_results.csv')
score = accuracy_score(y_test, rf_pred)
score 

<ipython-input-2-2382d21a0076>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


0.7330508474576272

In [4]:
import numpy as np

# Number of trees in random forest
n_estimators = np.linspace(100, 3000, int((3000-100)/200) + 1, dtype=int)
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [1, 5, 10, 20, 50, 75, 100, 150, 200]
# Minimum number of samples required to split a node
# min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 10, num = 9)]
min_samples_split = [1, 2, 5, 10, 15, 20, 30]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Criterion
criterion=['gini', 'entropy']
random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

random_grid

{'n_estimators': array([ 100,  307,  514,  721,  928, 1135, 1342, 1550, 1757, 1964, 2171,
        2378, 2585, 2792, 3000]),
 'max_depth': [1, 5, 10, 20, 50, 75, 100, 150, 200],
 'min_samples_split': [1, 2, 5, 10, 15, 20, 30],
 'min_samples_leaf': [1, 2, 3, 4],
 'bootstrap': [True, False],
 'criterion': ['gini', 'entropy']}

In [5]:
from sklearn.model_selection import RandomizedSearchCV

rf_base = RandomForestClassifier(random_state = 42)
rf_random = RandomizedSearchCV(estimator = rf_base,
                               param_distributions = random_grid,
                               n_iter = 30, cv = 5,
                               random_state=2, n_jobs = 4)

rf_random.fit(X_train, y_train)

C:\Users\Pooja\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.72324665 0.71900822 0.72748508 0.69354385 0.69567714 0.70733986
 0.72854891 0.72111899 0.72217719 0.6447484  0.72538557        nan
 0.70627603        nan 0.72431048 0.71158955 0.70310706 0.72324102
 0.71793876 0.71791062 0.72536305 0.71688056 0.72218845 0.72641563
 0.71262524 0.72324665 0.72853766 0.70415963 0.65006192 0.72217719]
  warnings.warn(
C:\Users\Pooja\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=30, n_jobs=4,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 5, 10, 20, 50, 75, 100,
                                                      150, 200],
                                        'min_samples_leaf': [1, 2, 3, 4],
                                        'min_samples_split': [1, 2, 5, 10, 15,
                                                              20, 30],
                                        'n_estimators': array([ 100,  307,  514,  721,  928, 1135, 1342, 1550, 1757, 1964, 2171,
       2378, 2585, 2792, 3000])},
                   random_state=2)

In [30]:
rf_grid_cv_pred = rf_random.predict(X_test)
test_df['rf_grid_cv_pred'] = rf_grid_cv_pred
test_df.to_csv('classification_results.csv')
score = accuracy_score(y_test, rf_grid_cv_pred)
score 

0.7288135593220338

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('insurance.csv')

cat_col = ['gender','region','smoker']
data_dummies = pd.get_dummies(data,columns = cat_col, drop_first = True)
X = data_dummies[['age','bmi','children','gender_male','region_northwest','region_southeast','region_southwest','smoker_yes']]
Y = data[['charges']]

# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
test_df = pd.concat ([X_test,y_test],axis =1 )

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)  
regressor.fit(X_train, y_train)  

rfreg_pred = regressor.predict(X_test)
test_df['rfreg_pred'] = rfreg_pred
test_df.to_csv('rf_reg_results.csv')
mse = mean_squared_error(y_test, rfreg_pred)
print(mse)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


20988954.421747707


In [38]:
import numpy as np
#hyeperparameters-list,boolean,int
n_estimators = np.linspace(100, 3000, int((3000-100)/200) + 1, dtype=int) #there's a list->100-3000 is the range,3rd parameter is 15
#which means there will be 15 values from 100-3000
max_features = ['auto', 'sqrt']
max_depth = [1, 5, 10, 20, 50, 75, 100, 150, 200]
min_samples_split = [1, 2, 5, 10, 15, 20, 30]
min_samples_leaf = [1, 2, 3, 4]
bootstrap = [True, False]
criterion=['mse', 'rmse']

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}


rf_base = RandomForestRegressor(random_state = 42)
rf_random = RandomizedSearchCV(estimator = rf_base,
                               param_distributions = random_grid,
                               n_iter = 30, cv = 5,
                               random_state=2, n_jobs = 4) # n_jobs runs parallely 

rf_random.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [40]:
rf_grid_reg_pred = rf_random.predict(X_test)
test_df['rf_grid_reg_pred'] = rf_grid_reg_pred
test_df.to_csv('rf_reg_results.csv')
mse = mean_squared_error(y_test, rf_grid_reg_pred)
print(mse)

19493857.645719808
